In [31]:
import pandas as pd
import numpy as np
import torch

In [126]:
ends = pd.read_csv("/Users/brentkong/Documents/curling/data/Ends.csv")
games = pd.read_csv("/Users/brentkong/Documents/curling/data/Games.csv")
stones = pd.read_csv("/Users/brentkong/Documents/curling/data/Stones.csv")

In [127]:
games["GameUID"] = games.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
stones["GameUID"] = stones.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
stones["EndUID"] = stones.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.TeamID)}_{int(r.EndID)}", axis=1)
ends.loc[ends["Result"] == 9, "Result"] = 0
ends["GameUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
ends["TeamUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.TeamID)}", axis=1)
ends["EndUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.TeamID)}_{int(r.EndID)}", axis=1)
ends["PowerPlay"] = ends["PowerPlay"].fillna(0)
ends["PowerPlayBool"] = ends["PowerPlay"].replace(2, 1)
ends = ends.sort_values(["GameUID", "TeamUID", "EndID"])
ends["CumulativeScore"] = ends.groupby("TeamUID")["Result"].cumsum()
stones = stones.merge(ends[["EndUID", "Result", "PowerPlay"]], on = "EndUID", how = "left").reset_index(drop = True)
ends = ends.reset_index(drop = True)

In [128]:
def assign_hammer(row):
    if row.ShotID % 2 != 0:
        return 0
    else:
        return 1

stones["Has_Hammer"] = stones.apply(assign_hammer, axis = 1)
ends = ends.merge(stones[["EndUID", "Has_Hammer"]], on="EndUID", how="left").drop_duplicates().reset_index(drop = True)

In [129]:
ends = ends.merge(games[["GameUID","NOC1","NOC2","TeamID1","TeamID2","LSFE"]], on="GameUID", how="left")

def get_opp_id(row):
    if row.TeamID == row.TeamID1:
        return row.TeamID2
    if row.TeamID == row.TeamID2:
        return row.TeamID1
    return None

ends["OpponentID"] = ends.apply(get_opp_id, axis=1)

In [130]:
ends["OpponentEndUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.OpponentID)}_{int(r.EndID)}", axis=1)

ends = ends.merge(
    ends[["EndUID", "CumulativeScore"]].rename(columns={
        "EndUID": "OpponentEndUID",
        "CumulativeScore": "OpponentCumulative"
    }),
    on="OpponentEndUID",
    how="left"
)

ends["ScoreDiff"] = ends["CumulativeScore"] - ends["OpponentCumulative"]


ends = ends.sort_values(["GameUID", "TeamID", "EndID"])

ends["PrevScoreDiff"] = (
    ends.groupby(["GameUID", "TeamID"])["ScoreDiff"]
        .shift(1)
        .fillna(0)
)


In [131]:
ends[ends["EndID"] == 1]["PrevScoreDiff"].value_counts()

PrevScoreDiff
0.0    688
Name: count, dtype: int64

In [132]:
ends = ends.merge(
    ends[["EndUID", "Result"]].rename(columns={
        "EndUID": "OpponentEndUID",
        "Result": "OpponentResult"
    }),
    on="OpponentEndUID",
    how="left"
)
ends["EndDiff"] = ends["Result"] - ends["OpponentResult"]

ends = ends.sort_values(["GameUID", "TeamID", "EndID"])

ends["PrevEndDiff"] = (
    ends.groupby(["GameUID", "TeamID"])["EndDiff"]
        .shift(1)
        .fillna(0)
)



In [133]:
ends[ends["EndID"] == 1]["PrevEndDiff"].value_counts()

PrevEndDiff
0.0    688
Name: count, dtype: int64

In [134]:
ends.columns

Index(['CompetitionID', 'SessionID', 'GameID', 'TeamID', 'EndID', 'Result',
       'PowerPlay', 'GameUID', 'TeamUID', 'EndUID', 'PowerPlayBool',
       'CumulativeScore', 'Has_Hammer', 'NOC1', 'NOC2', 'TeamID1', 'TeamID2',
       'LSFE', 'OpponentID', 'OpponentEndUID', 'OpponentCumulative',
       'ScoreDiff', 'PrevScoreDiff', 'OpponentResult', 'EndDiff',
       'PrevEndDiff'],
      dtype='object')

In [135]:
out = {}
for end in range(1, 9):
    value_counts = ends.loc[ends["EndID"] == end]["EndDiff"].value_counts().to_dict()
    for key in value_counts:
        value_counts[key] /= len(ends.loc[ends["EndID"] == end])
    out[end] = value_counts
print(out)

{1: {1: 0.27906976744186046, -1: 0.27906976744186046, 2: 0.1308139534883721, -2: 0.1308139534883721, 3: 0.055232558139534885, -3: 0.055232558139534885, -4: 0.027616279069767442, 4: 0.027616279069767442, 5: 0.005813953488372093, -5: 0.005813953488372093, 0: 0.0029069767441860465}, 2: {-1: 0.28488372093023256, 1: 0.28488372093023256, -2: 0.1380813953488372, 2: 0.1380813953488372, -3: 0.05377906976744186, 3: 0.05377906976744186, -4: 0.0188953488372093, 4: 0.0188953488372093, 5: 0.00436046511627907, -5: 0.00436046511627907}, 3: {-1: 0.32122093023255816, 1: 0.32122093023255816, -2: 0.125, 2: 0.125, 3: 0.0377906976744186, -3: 0.0377906976744186, 4: 0.011627906976744186, -4: 0.011627906976744186, 5: 0.0029069767441860465, -5: 0.0029069767441860465, 0: 0.0029069767441860465}, 4: {-1: 0.28343023255813954, 1: 0.28343023255813954, -2: 0.14244186046511628, 2: 0.14244186046511628, 3: 0.05377906976744186, -3: 0.05377906976744186, -4: 0.015988372093023256, 4: 0.015988372093023256, 0: 0.00290697674418

In [121]:
ends

,CompetitionID,SessionID,GameID,TeamID,EndID,Result,PowerPlay,GameUID,TeamUID,EndUID,...,TeamID2,LSFE,OpponentID,OpponentEndUID,OpponentCumulative,ScoreDiff,PrevScoreDiff,OpponentResult,EndDiff,PrevEndDiff
0,0,10,1,20,1,1,0.0,0_10_1,0_10_1_20,0_10_1_20_1,...,22,0,22,0_10_1_22_1,0,1,0.0,0,1,0.0
1,0,10,1,20,2,0,0.0,0_10_1,0_10_1_20,0_10_1_20_2,...,22,0,22,0_10_1_22_2,3,-2,1.0,3,-3,1.0
2,0,10,1,20,3,3,0.0,0_10_1,0_10_1_20,0_10_1_20_3,...,22,0,22,0_10_1_22_3,3,1,-2.0,0,3,-3.0
3,0,10,1,20,4,0,0.0,0_10_1,0_10_1_20,0_10_1_20_4,...,22,0,22,0_10_1_22_4,7,-3,1.0,4,-4,3.0
4,0,10,1,20,5,3,0.0,0_10_1,0_10_1_20,0_10_1_20_5,...,22,0,22,0_10_1_22_5,7,0,-3.0,0,3,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,24250026,9,5,25,4,1,0.0,24250026_9_5,24250026_9_5_25,24250026_9_5_25_4,...,25,0,12,24250026_9_5_12_4,3,-1,-2.0,0,1,-1.0
5276,24250026,9,5,25,5,0,0.0,24250026_9_5,24250026_9_5_25,24250026_9_5_25_5,...,25,0,12,24250026_9_5_12_5,4,-2,-1.0,1,-1,1.0
5277,24250026,9,5,25,6,0,1.0,24250026_9_5,24250026_9_5_25,24250026_9_5_25_6,...,25,0,12,24250026_9_5_12_6,5,-3,-2.0,1,-1,-1.0
5278,24250026,9,5,25,7,0,0.0,24250026_9_5,24250026_9_5_25,24250026_9_5_25_7,...,25,0,12,24250026_9_5_12_7,6,-4,-3.0,1,-1,-1.0


In [137]:
pp_varient = {"PP": ends.loc[ends["PowerPlay"] == 0],  "NO_PP": ends.loc[ends["PowerPlay"] == 0]}
final = {}
for action, variant in pp_varient.items():
    out = {}
    for end in range(1, 9):
        value_counts = variant.loc[variant["EndID"] == end]["EndDiff"].value_counts().to_dict()
        for key in value_counts:
            value_counts[key] /= len(variant.loc[variant["EndID"] == end])
        out[end] = value_counts
    final[action] = out
print(final)

{'PP': {1: {1: 0.27906976744186046, -1: 0.27906976744186046, 2: 0.1308139534883721, -2: 0.1308139534883721, 3: 0.055232558139534885, -3: 0.055232558139534885, -4: 0.027616279069767442, 4: 0.027616279069767442, 5: 0.005813953488372093, -5: 0.005813953488372093, 0: 0.0029069767441860465}, 2: {1: 0.2852983988355167, -1: 0.2838427947598253, -2: 0.13828238719068414, 2: 0.13828238719068414, -3: 0.053857350800582245, 3: 0.053857350800582245, -4: 0.018922852983988356, 4: 0.018922852983988356, 5: 0.004366812227074236, -5: 0.004366812227074236}, 3: {-1: 0.32392273402674593, 1: 0.3179791976225854, 2: 0.1263001485884101, -2: 0.12481426448736999, -3: 0.03863298662704309, 3: 0.03566121842496285, 4: 0.01188707280832095, -4: 0.01188707280832095, 5: 0.0029717682020802376, -5: 0.0029717682020802376, 0: 0.0029717682020802376}, 4: {-1: 0.29046898638426627, 1: 0.2753403933434191, -2: 0.1437216338880484, 2: 0.14069591527987896, 3: 0.05446293494704992, -3: 0.0529500756429652, -4: 0.01664145234493192, 4: 0.01

In [ ]:
out = {}
for end in range(1, 9):
    value_counts = ends.loc[ends["EndID"] == end]["EndDiff"].value_counts().to_dict()
    for key in value_counts:
        value_counts[key] /= len(ends.loc[ends["EndID"] == end])
    out[end] = value_counts
print(out)

In [101]:
ends_df = ends[
    ['EndUID', "Has_Hammer",
            "PowerPlayBool",
            "EndID",
            "PrevScoreDiff"]
    ]


'''
games_df = games[[
    "GameUID", "NOC1",	"NOC2",	"ResultStr1", "ResultStr2",	"LSFE",	"Winner",
    "TeamID1", "TeamID2",
]]


stones_df = stones[[
    'GameUID', 'TeamID', 'EndID', 'ShotID', 'PlayerID', 'Task', 'Handle', 'Points', 'stone_1_x',
    'stone_1_y', 'stone_2_x', 'stone_2_y', 'stone_3_x', 'stone_3_y',
    'stone_4_x', 'stone_4_y', 'stone_5_x', 'stone_5_y', 'stone_6_x',
    'stone_6_y', 'stone_7_x', 'stone_7_y', 'stone_8_x', 'stone_8_y',
    'stone_9_x', 'stone_9_y', 'stone_10_x', 'stone_10_y', 'stone_11_x',
    'stone_11_y', 'stone_12_x', 'stone_12_y',
    'Has_Hammer', "PowerPlay", 'Result', 'BurialDepth', 'GuardAngle', 'ClusterIndex',
    'SideOpenness', 
]]
'''
ends_df.to_csv("/Users/brentkong/Documents/curling/processed_data/ends_processed_bayesian.csv", index=False)
#games_df.to_csv("/Users/brentkong/Documents/curling/processed_data/games_processed.csv", index=False)
#stones_df.to_csv("/Users/brentkong/Documents/curling/processed_data/stones_processed.csv", index=False)